In [1]:
import pandas as pd
from diskcache import Cache
from pathlib import Path
import requests
import json
import os
import re

Definition et creation des fonctions pour l'embeddings a partir de l'API JDM

Mettre les generiques + les infos semantiques (relation 36) + marqueurs de type (numero de la relation)

Pour le vecteur B regarder l'article (determine ou non determine)

FAIRE DES PRODUITS SCALAIRES SUR DES VECTEURS NORMES


Useful links :

https://www.jeuxdemots.org/jdm-about.php

https://jdm-api.demo.lirmm.fr/schema

https://jdm-api.demo.lirmm.fr/v0/relations/from/{node1_id}

https://jdm-api.demo.lirmm.fr/v0/relations/to/{node2_name}

https://jdm-api.demo.lirmm.fr/v0/node_by_name/{node_name}

In [22]:
CACHE_DIR = "./.jdm_cache"
#relations_path = Path("/home/mario/Desktop/portfolio_projects/natural_language_semantics/relations")

# path matt
relations_path = Path("C:/Users/Matthieu/Documents/Fac/M2_IASD/TALN/projet_notebook/genitive_semantic_predition/relations")

In [23]:
cache = Cache()

In [44]:
def load_relations(directory_path: str):

    rows = []

    for filename in os.listdir(directory_path):
        full_path = os.path.join(directory_path, filename)
        relation_num = int(re.split("[-.]",filename)[0])
        if os.path.isfile(full_path):
            
            with open(full_path, "r", encoding="utf-8") as f:
                data = f.read()
                data = re.split("\n",data)
                
                for l in data:
                     rows.append({"content": l, "filename": relation_num})

           

    df = pd.DataFrame(rows)

    print(df.head())
    

    return df

In [64]:
def get_relations_JDM(node_name: str):
    with cache as c:
        if c.__contains__(node_name):
            return c.get(node_name)
        else:
            res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/to/{node_name}")
            res = json.loads(res.text)
            c.set(node_name,res)
            return res

In [46]:
df = load_relations(relations_path)

                       content  filename
0           le vin de Bordeaux         1
1         le fromage de chèvre         1
2  l’huile d’olive de Provence         1
3           le sable du Sahara         1
4            le café du Brésil         1


In [60]:
def strip_articles(s):
    if not s:
        return s
    s = s.replace("’", "'").strip()
    s = re.sub(r"^(les|la|le|une|un|l')\s?", "", s, flags=re.IGNORECASE)
    s = re.sub(r"\b(de|d|à|en|sur|pour)\s+l'", r"\1 ", s, flags=re.IGNORECASE)

    return s.strip()


In [61]:
df.iloc[:50,0].apply(lambda x: strip_articles(x))

0               vin de Bordeaux
1             fromage de chèvre
2     huile d'olive de Provence
3               sable du Sahara
4                café du Brésil
5                 thé de Ceylan
6                 soie de Chine
7                 épices d'Inde
8              miel de montagne
9                 eau de source
10              laine de mouton
11              marbre d'Italie
12                 riz du Japon
13                cuir de vache
14               coton d'Égypte
15               pierre du Jura
16                bois de chêne
17             parfum de Grasse
18            tabac de Virginie
19               sucre de canne
20    chocolat de Côte d'Ivoire
21                   sel de mer
22                  vin du Rhin
23               pain de seigle
24              miel de lavande
25                farine de blé
26            cuir de crocodile
27             métal du minerai
28          poussière d'étoiles
29                 eau du puits
30                 vent du nord
31      

In [62]:
embedding = dict()
embedding["info_sem"] = []
embedding["type_marqueur"] = set()
embedding["hyperonyme"] = []

In [105]:
# obtenir les relations d'un mot
#res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/to/{node_name}")
res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/to/matelas")

In [106]:
# parser le json
res = json.loads(res.text)

In [107]:
res["nodes"]

[{'id': 237, 'name': 'objet', 'type': 1, 'w': 9403},
 {'id': 299, 'name': 'lit-cage', 'type': 1, 'w': 74},
 {'id': 855, 'name': 'regonflé', 'type': 1, 'w': 54},
 {'id': 978, 'name': 'affaissé', 'type': 1, 'w': 50},
 {'id': 1313, 'name': 'couchure', 'type': 1, 'w': 50},
 {'id': 1320, 'name': 'réserves', 'type': 1, 'w': 3207},
 {'id': 1353, 'name': 'se dégonfler', 'type': 1, 'w': 60},
 {'id': 1990, 'name': 'somnoler', 'type': 1, 'w': 189},
 {'id': 2666, 'name': 'coucher', 'type': 1, 'w': 3615},
 {'id': 2685, 'name': 'duvetine', 'type': 1, 'w': 50},
 {'id': 2849, 'name': 'layetterie', 'type': 1, 'w': 50},
 {'id': 2910, 'name': 'schloff', 'type': 1, 'w': 50},
 {'id': 3264, 'name': 'alèse', 'type': 1, 'w': 98},
 {'id': 3300, 'name': 'matelassé', 'type': 1, 'w': 96},
 {'id': 3469, 'name': 'natter', 'type': 1, 'w': 79},
 {'id': 4322, 'name': 'tapissier', 'type': 1, 'w': 69},
 {'id': 5430, 'name': 'housse', 'type': 1, 'w': 98},
 {'id': 5435, 'name': 'ronfler', 'type': 1, 'w': 326},
 {'id': 569

In [108]:
# filtrer les noeuds pour les relations voulues + les poids negatifs
for node in res["nodes"]:
    
    
    if(node["w"] <= 0 or node["type"] == 200): continue
    
    if(node["type"] == 36):
        embedding["info_sem"].append(node["id"])
    elif(node["type"] == 6):
        embedding["hyperonyme"].append(node["id"])
    
    embedding["type_marqueur"].add(node["type"])

In [109]:
embedding["hyperonyme"]

[]

In [110]:
# utiliser le nombre de fois que la relation est presente comme poids ?
embedding["type_marqueur"]

{1, 2, 8, 9, 12, 666, 777}

In [111]:
embedding["info_sem"]

[]